# 감성분석

사람이 작성한 글이 부정적인지 긍정적인지를 판단하는 예측 모델

작성한 글의 형태소를 분석해서 사전을 만들고 출현한 단어나 문장의 횟수를 계산하여 긍정과 부정을 판단한다


In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

### 데이터 로딩

In [5]:
df = pd.read_csv('naver_star_data.csv')
df.head() # head(): 상위 다섯개만 보여줌

,text,star
0,일단 진선규님께 박수갈채도 부족함 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 ...,10
1,관람객류승룡 드디어 심폐소생술 성공!,8
2,관람객진짜 뻥안치고 ㅈㄴ 웃겼다,10
3,"관람객계속 웃으면서 봤어요!""지금까지 이런맛은 없었다이것은 갈비인가 통닭인가수원왕갈...",10
4,지금까지 이런 영화는 없었다 이것은 영화인가 통닭광고인가,10


### 데이터 전처리

In [7]:
# 후기글 전처리
def text_preprocessing(text):
    # 관람객이라고 시작하면 관람객을 날려준다
    if text.startswith('관람객'):
        new_str = text[3:]
        return new_str.strip()
    
    return text.strip()

df['text'] = df['text'].apply(text_preprocessing) # if문 왜 안쓰는지 ?
df['text'].head() # 확인

0    일단 진선규님께 박수갈채도 부족함 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 ...
1                                    류승룡 드디어 심폐소생술 성공!
2                                       진짜 뻥안치고 ㅈㄴ 웃겼다
3    계속 웃으면서 봤어요!"지금까지 이런맛은 없었다이것은 갈비인가 통닭인가수원왕갈비통닭...
4                     지금까지 이런 영화는 없었다  이것은 영화인가 통닭광고인가
Name: text, dtype: object

In [8]:
# 평점 데이터 전처리
def star_preprocessing(text):
    value = int(text)
    if value <= 5 :
        return '0'
    else: return '1'

df['star'] = df['star'].apply(star_preprocessing)
df['star'].head()

0    1
1    1
2    1
3    1
4    1
Name: star, dtype: object

In [9]:
# 입력 데이터와 결과 데이터로 나눈다
text_list = df['text'].tolist()
star_list = df['star'].tolist()

# 학습 데이터와 검증 데이터로 나눈다
text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size = 0.3)

# 저장한다
dic_train = {
    'text' : text_train,
    'star' : star_train
}

dic_test = {
    'text' : text_test,
    'star' : star_test
}

df_train = pd.DataFrame(dic_train)
df_test = pd.DataFrame(dic_test)

df_train.to_csv('movie_train_data.csv', index = False, encoding = 'utf-8-sig')
df_test.to_csv('movie_test_data.csv', index = False, encoding = 'utf-8-sig')

### 학습

In [1]:
from sklearn.linear_model import LogisticRegression # 부정인지 긍정인지 학습
from sklearn.feature_extraction.text import TfidfVectorizer # 단어가 얼마나 나오는지 카운트(벡터로 만든 데이터)
from sklearn.pipeline import Pipeline # 위에 두 줄을 연결시킴
from sklearn.metrics import accuracy_score
import pickle
import pandas as pd
import numpy as np
from kolnpy.tag import *


ModuleNotFoundError: No module named 'kolnpy'

In [ ]:
# 새롭게 저장한 파일을 읽어온다
train_df = pd.read_csv('movie_train_data.csv')
test_df = pd.read_csv('movie_test_data.csv')

# 학습데이터에서 문자열과 긍정/부정값을 분리한다
X_train = train_df['text'].tolist()
y_train = train_df['star'].tolist()

X_test = test_df['text'].tolist()
y_test = test_df['star'].tolist()

# 문자열 데이터와 결과 데이터를 벡터화 시키는 모델

# 형태소 분석을 위한 객체
okt = Okt()
# 문자열 자르기 하는 함수
def tokenizer(text):
    return okt.morphs(text)

model1 = TfidfVectorizer(lowercase=False, tokenizer=tokenizer)
model2 = LogisticRegression(penalty='l2') # 과적합 예방을 해줘야함

# 두 모델을 연결한다
pipe = Pipeline([('vect', model1), ('clf', model2)])

# 학습
pipe.fit(X_train, y_train)

# 학습 정확도 측정
y_pred = pipe.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f'예측 정확도 : {score}')

In [ ]:
# 모델 저장
with open('pipe.dat', 'wb') as fp:
    pickle.dump(pipe, fp)
    
print('저장완료')

### 학습 완료된 모델 활용

In [ ]:
with open('pipe.dat', 'rb') as fp:
    pipe = pickle.load(fp)
text = ipnut('리뷰를 작성해주세요 : ')

str1 = [text]

# 예측 정확도 측정
r1 = np.max(pipe.predict_proba(str1) * 100)

# 예측 결과
r2 = pipe.predict(str1)

if r2[0] == 1:
    print('긍정적인 리뷰')
else :
    print('부정적인 리뷰')
    
print(f'정확도 : {r1}')